In [1]:
import os
os.chdir("..")

In [40]:
import pandas as pd
import re
from scripts.Common.DB_Utils import DBInstance
import unicodedata

In [4]:
path=r"/app/data/raw/Classes/Classes.xlsx"

In [45]:
df=pd.read_excel(path, sheet_name='Disciplinas')

In [46]:
ajuste=["OBJETIVOS", "METODOLOGIA EXTENSIONISTA", "EMENTA"]

In [47]:
def format_text(text):
    text = re.sub(r'\t+', '', str(text))      # remove tabs
    text = re.sub(r'\n+', '\n', text)         # colapsa múltiplas quebras de linha
    return text.strip()

In [48]:
for i in ajuste:
    df[i] = df[i].apply(format_text)

In [49]:
def normalize_column(col):
    
    # Remove acentos
    col = unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8')
    
    #espaço por underscore
    col = re.sub(r"\s", "_", col).lower()

    return col

In [50]:
df.columns=list(map(normalize_column, df.columns))

In [51]:
df.columns

Index(['sigla', 'disciplina', 'tpei', 'recomendacao', 'objetivos',
       'metodologia_extensionista', 'ementa', 'bibliografia_basica',
       'bibliografia_complementar'],
      dtype='object')

In [38]:
df[["objetivos", "metodologia_extensionista", "ementa"]][df["sigla"]=='NHBB003-23']

objetivos  \
1  Protagonizar ações de extensão utilizando temas da grande área da biodiversidade e evolução. Produzir um resultado concreto da ação extensionista que responda as demandas levandas levantadas pelo público alvo e possa ser disponibilizado para o público não universitário e não científico.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

# <span font style="color:blue">Basic exportation tests</span>

In [52]:
db_instance=DBInstance()

In [53]:
engine=db_instance.get_engine()

In [54]:
df.to_sql('classes', con=engine, if_exists='append', index=False, schema="bronze")

354

# <span font style="color:blue">Basic bibliography tests</span>

In [7]:
pd.set_option('display.max_colwidth', None)

In [20]:
df_books=df[["SIGLA", "BIBLIOGRAFIA BÁSICA"]][df["SIGLA"]=='ESZR007-21']

In [21]:
df_books

,SIGLA,BIBLIOGRAFIA BÁSICA
0,ESZR007-21,"KASSENOVA, Togzhan. Brazil’s Nuclear Kaleidoscope: an envolving identity. Washington: Carnegie Endowment for International Peace, 2014. Disponível em: https://carnegieendowment.org/files/brazil_nuclear_kaleidoscope_lo_res.pdf \nMARZO, M.; ALMEIDA, S. A Evolução do Controle de Armas Nucleares: Desarmamento e Não Proliferação. Rio de Janeiro: Ciência Moderna, 2006. \nPERCOVICH, George; ACTON, James (eds.) Abolishing Nuclear Weapons: A Debate. Carnegie Endowment for International Peace. Washington: VMW Printing Inc., 2009. Disponível em: https://carnegieendowment.org/files/abolishing_nuclear_weapons_debate.pdf"


In [22]:
df_books["BIBLIOGRAFIA BÁSICA"]=df_books["BIBLIOGRAFIA BÁSICA"].str.split('\n')

In [23]:
df_books

,SIGLA,BIBLIOGRAFIA BÁSICA
0,ESZR007-21,"[KASSENOVA, Togzhan. Brazil’s Nuclear Kaleidoscope: an envolving identity. Washington: Carnegie Endowment for International Peace, 2014. Disponível em: https://carnegieendowment.org/files/brazil_nuclear_kaleidoscope_lo_res.pdf , MARZO, M.; ALMEIDA, S. A Evolução do Controle de Armas Nucleares: Desarmamento e Não Proliferação. Rio de Janeiro: Ciência Moderna, 2006. , PERCOVICH, George; ACTON, James (eds.) Abolishing Nuclear Weapons: A Debate. Carnegie Endowment for International Peace. Washington: VMW Printing Inc., 2009. Disponível em: https://carnegieendowment.org/files/abolishing_nuclear_weapons_debate.pdf ]"


In [26]:
df_books=df_books.explode('BIBLIOGRAFIA BÁSICA')

In [27]:
df_books

,SIGLA,BIBLIOGRAFIA BÁSICA
0,ESZR007-21,"KASSENOVA, Togzhan. Brazil’s Nuclear Kaleidoscope: an envolving identity. Washington: Carnegie Endowment for International Peace, 2014. Disponível em: https://carnegieendowment.org/files/brazil_nuclear_kaleidoscope_lo_res.pdf"
0,ESZR007-21,"MARZO, M.; ALMEIDA, S. A Evolução do Controle de Armas Nucleares: Desarmamento e Não Proliferação. Rio de Janeiro: Ciência Moderna, 2006."
0,ESZR007-21,"PERCOVICH, George; ACTON, James (eds.) Abolishing Nuclear Weapons: A Debate. Carnegie Endowment for International Peace. Washington: VMW Printing Inc., 2009. Disponível em: https://carnegieendowment.org/files/abolishing_nuclear_weapons_debate.pdf"


## <b>Complementary bibliography</b>

In [31]:
df_books_2=df[["SIGLA", "BIBLIOGRAFIA COMPLEMENTAR"]][df["SIGLA"]=='ESZR007-21']

In [32]:
df_books_2

,SIGLA,BIBLIOGRAFIA COMPLEMENTAR
0,ESZR007-21,"CAVES, John.; CARUS, Seth. The Future of Weapons of Mass Destruction: Their Nature and Role in 2030. Center for the Study of Weapons of Mass Destruction. Ocassional paper No 10. Wasthington: National Defense University Print, 2014. Disponível em: https://www.hsdl.org/?abstract&did=755104\nKREB, Krestin. “La Política Nuclear en América Latina. Breve Análisis sobre el uso de la Energia Nuclear en América Latina.” Policy Paper 16. Fundação Friedrich Ebert, FES-ILDIS. Quito: Ofset Gráficas, 2011. Disponível em: http://library.fes.de/pdf-files/bueros/quito/08677.pdf\nPRICE, Owen.; MACKBY Jenifer (Eds). “Debating 21st Century Nuclear Issues. Center for Strategic and International Studies” Washington, D.C, 2007. Disponível em: https://csis-website- prod.s3.amazonaws.com/s3fs-public/legacy_files/files/media/csis/pubs/07-09-14_priceponi.pdf\nSCHEIDER, M.; FROGATT, A. (Editors). The World Nuclear Industry Status Report 2018. Paris – London. Disponível em: https://www.worldnuclearreport.org/IMG/pdf/20180902wnisr2018-hr.pdf\nVIENA CENTER FOR DISARMAMENT AND NON PROLIFERATION. Cooperation among Nuclear-Weapon-Free Zones: History, Challenges and Recommendations. VCDNP Task Force Report. March, 2018. Disponível em: https://vcdnp.org/wp-content/uploads/2018/03/NWFZ-TF- Report-final-1.pdf"


In [33]:
df_books_2["BIBLIOGRAFIA COMPLEMENTAR"]=df_books_2["BIBLIOGRAFIA COMPLEMENTAR"].str.split(r'\n')

In [34]:
df_books_2

,SIGLA,BIBLIOGRAFIA COMPLEMENTAR
0,ESZR007-21,"[CAVES, John.; CARUS, Seth. The Future of Weapons of Mass Destruction: Their Nature and Role in 2030. Center for the Study of Weapons of Mass Destruction. Ocassional paper No 10. Wasthington: National Defense University Print, 2014. Disponível em: https://www.hsdl.org/?abstract&did=755104, KREB, Krestin. “La Política Nuclear en América Latina. Breve Análisis sobre el uso de la Energia Nuclear en América Latina.” Policy Paper 16. Fundação Friedrich Ebert, FES-ILDIS. Quito: Ofset Gráficas, 2011. Disponível em: http://library.fes.de/pdf-files/bueros/quito/08677.pdf, PRICE, Owen.; MACKBY Jenifer (Eds). “Debating 21st Century Nuclear Issues. Center for Strategic and International Studies” Washington, D.C, 2007. Disponível em: https://csis-website- prod.s3.amazonaws.com/s3fs-public/legacy_files/files/media/csis/pubs/07-09-14_priceponi.pdf, SCHEIDER, M.; FROGATT, A. (Editors). The World Nuclear Industry Status Report 2018. Paris – London. Disponível em: https://www.worldnuclearreport.org/IMG/pdf/20180902wnisr2018-hr.pdf, VIENA CENTER FOR DISARMAMENT AND NON PROLIFERATION. Cooperation among Nuclear-Weapon-Free Zones: History, Challenges and Recommendations. VCDNP Task Force Report. March, 2018. Disponível em: https://vcdnp.org/wp-content/uploads/2018/03/NWFZ-TF- Report-final-1.pdf ]"


In [35]:
df_books_2=df_books_2.explode('BIBLIOGRAFIA COMPLEMENTAR')

In [30]:
df_books_2

,SIGLA,BIBLIOGRAFIA COMPLEMENTAR
0,ESZR007-21,"CAVES, John.; CARUS, Seth. The Future of Weapons of Mass Destruction: Their Nature and Role in 2030. Center for the Study of Weapons of Mass Destruction. Ocassional paper No 10. Wasthington: National Defense University Print, 2014. Disponível em: https://www.hsdl.org/?abstract&did=755104"
0,ESZR007-21,"KREB, Krestin. “La Política Nuclear en América Latina. Breve Análisis sobre el uso de la Energia Nuclear en América Latina.” Policy Paper 16. Fundação Friedrich Ebert, FES-ILDIS. Quito: Ofset Gráficas, 2011. Disponível em: http://library.fes.de/pdf-files/bueros/quito/08677.pdf"
0,ESZR007-21,"PRICE, Owen.; MACKBY Jenifer (Eds). “Debating 21st Century Nuclear Issues. Center for Strategic and International Studies” Washington, D.C, 2007. Disponível em: https://csis-website- prod.s3.amazonaws.com/s3fs-public/legacy_files/files/media/csis/pubs/07-09-14_priceponi.pdf"
0,ESZR007-21,"SCHEIDER, M.; FROGATT, A. (Editors). The World Nuclear Industry Status Report 2018. Paris – London. Disponível em: https://www.worldnuclearreport.org/IMG/pdf/20180902wnisr2018-hr.pdf"
0,ESZR007-21,"VIENA CENTER FOR DISARMAMENT AND NON PROLIFERATION. Cooperation among Nuclear-Weapon-Free Zones: History, Challenges and Recommendations. VCDNP Task Force Report. March, 2018. Disponível em: https://vcdnp.org/wp-content/uploads/2018/03/NWFZ-TF- Report-final-1.pdf"


# <span font style="color:blue"> Exportation tests - Bibliography</span>

In [62]:
def Bibliography_Transformation(
                                df_arg: pd.DataFrame, 
                                Column: str,
                                sep: str
                                ) -> pd.DataFrame:

    df_readings=df_arg[["SIGLA", Column]]
    df_readings[Column]=df_readings[Column].str.split(sep)

    return df_readings.explode(Column)    

In [63]:
df_books_basic=Bibliography_Transformation(df, "BIBLIOGRAFIA BÁSICA", r"\n")
df_books_compl=Bibliography_Transformation(df, "BIBLIOGRAFIA COMPLEMENTAR", r"\n")

/tmp/ipykernel_926/3600396081.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_readings[Column]=df_readings[Column].str.split(sep)
/tmp/ipykernel_926/3600396081.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_readings[Column]=df_readings[Column].str.split(sep)


In [47]:
db_instance=DBInstance()
engine=db_instance.get_engine()

In [48]:
df_books_basic.to_sql('Biliography_Basic', con=engine, if_exists='append', index=False, schema="bronze")
df_books_compl.to_sql('Biliography_Complementary', con=engine, if_exists='append', index=False, schema="bronze")

867

In [52]:
df_books_basic.to_sql('Biliography_Basic', con=engine, if_exists='append', index=False, schema="bronze")

308

# <span font style="color:blue">Dependencies</span>

In [60]:
df_dependencies=df[["SIGLA", "RECOMENDAÇÃO"]]

In [ ]:
df_dependencies["RECOMENDAÇÃO"]=df_dependencies["RECOMENDAÇÃO"].str

,SIGLA,RECOMENDAÇÃO
0,ESZR007-21,Não há
1,NHBB003-23,Evolução e Diversificação da Vida na Terra; Biodiversidade: Interações entre organismos e ambiente; Evolução
2,NHBB004-23,"Evolução e Diversificação da Vida na Terra; Bioquímica: estrutura, propriedade e funções de biomoléculas; Genética II; Biologia Celular; Bioquímica Funcional"
3,NHBB005-23,Evolução e Diversificação da Vida na Terra; Biodiversidade: Interações entre organismos e ambiente; Evolução e Diversidade de Plantas I; Evolução e Diversidade de Plantas II
4,NHBB006-23,Histologia e Embriologia; Biologia celular; Morfofisiologia Humana I; Morfofisiologia Humana II; Morfofisiologia Humana III
...,...,...
1349,MCZA052-22,Algoritmos e Estruturas de Dados I; Banco de Dados; Programação Orientada a Objetos
1350,MCZA031-13,Inteligência Artificial
1351,NHBB002-23,Sistemática e Biogeografia; Zoologia de Invertebrados I
1352,NHT1065-15,Sistemática e Biogeografia; Zoologia de Ecdysozoa


In [65]:
df_dependencies_TG=Bibliography_Transformation(df, "RECOMENDAÇÃO", r";")

/tmp/ipykernel_926/3600396081.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_readings[Column]=df_readings[Column].str.split(sep)


In [66]:
df_dependencies_TG

,SIGLA,RECOMENDAÇÃO
0,ESZR007-21,Não há
1,NHBB003-23,Evolução e Diversificação da Vida na Terra
1,NHBB003-23,Biodiversidade: Interações entre organismos e ambiente
1,NHBB003-23,Evolução
2,NHBB004-23,Evolução e Diversificação da Vida na Terra
...,...,...
1351,NHBB002-23,Sistemática e Biogeografia
1351,NHBB002-23,Zoologia de Invertebrados I
1352,NHT1065-15,Sistemática e Biogeografia
1352,NHT1065-15,Zoologia de Ecdysozoa


In [67]:
df_dependencies_TG.to_sql('Dependencies', con=engine, if_exists='append', index=False, schema="bronze")

414